# Step 1
Download "all results" "as displayed in the in data table"
from
https://stash.looker.com/explore/system__activity/history?qid=GIgQ1d1zTzbDW85pAZXWat
to `0000-looker.csv`.
Only includes last 90 days because looker only saves queries for 90 days.

# Step 2
Run the following.

In [1]:
! pip install -q lkml
! pip install -q --upgrade pip

import os
import sys

import lkml
import pandas as pd

LOOKER_DIR = '/Users/sunderland/Dropbox/stash22/workspace/stash-invest-looker'
print(f'\nChange directory\nfrom {os.getcwd()}\nto {LOOKER_DIR}\n')
os.chdir(LOOKER_DIR)


Change directory
from /Users/sunderland/Library/CloudStorage/Dropbox/stash22/workspace/data-science-sandbox/other_matt
to /Users/sunderland/Dropbox/stash22/workspace/stash-invest-looker



In [2]:
files = ! find .
model_files = [x[2:] for x in files if x[-11:] == '.model.lkml']
view_files = [x[2:] for x in files if x[-10:] == '.view.lkml']

In [3]:
explore_list = []

for model_file in model_files:
    model_name = model_file[:-11].split('/')[-1]
    with open(model_file, 'r') as f:
        model = lkml.load(f)
    for explore in model['explores']:
        explore_name = explore['name']
        for view_name in (
            {explore_name}
                | {join.get('from') or join['name']
                for join in explore.get('joins') or []}
        ):
            explore_list.append((model_name, explore_name, view_name))

explore_df = pd.DataFrame.from_records(explore_list)
explore_df.columns = ['model_name','explore_name','view_name']

In [4]:
view_list = []

for view_filename in view_files:
    with open(view_filename, 'r') as f:
        view_file = lkml.load(f)
    for view in view_file.get('views') or []:
        view_name = view['name']
        schema_table = view.get('sql_table_name')
        schema_name = schema_table.split('.')[0] if schema_table else 'DERIVED_TABLE'
        table_name = schema_table.split('.')[1] if schema_table else 'DERIVED_TABLE'
        view_list.append((view_name, schema_name, table_name))

view_df = pd.DataFrame.from_records(view_list)
view_df.columns = ['view_name','schema_name', 'table_name']

In [5]:
query_df = pd.read_csv('/Users/sunderland/Dropbox/stash22/workspace/data-science-sandbox/other_matt/0000-looker-input.csv')
query_df.columns = ['model_name', 'explore_name', 'explore_last_query_date', 'explore_query_count_90d']

In [6]:
df = explore_df.merge(
    query_df,
    left_on=['model_name','explore_name'],
    right_on=['model_name','explore_name'],
    how='outer',
).merge(
    view_df,
    left_on=['view_name'],
    right_on=['view_name'],
    how='outer',
)[[
    'model_name','explore_name','view_name',
    'schema_name','table_name',
    'explore_query_count_90d','explore_last_query_date',
]].sort_values(['model_name','explore_name','schema_name','table_name'], ascending=True)

OUTPUT_FILENAME = '/Users/sunderland/Dropbox/stash22/workspace/data-science-sandbox/other_matt/0000-looker.csv'
df.to_csv(OUTPUT_FILENAME, index=False)
print(f'\nSaved to {OUTPUT_FILENAME}\n')


Saved to /Users/sunderland/Dropbox/stash22/workspace/data-science-sandbox/other_matt/0000-looker.csv

